## IMC com Machine Learning

#### Por Bruno Luvizotto Carli

*Hello there*,

Você tem interesse ou está curioso sobre o crescente campo da Inteligência Artificial, que está se popularizando a cada dia mais?

Pretendo demonstrar nesse pequeno tutorial prático, como pode ser simples desenvolver uma aplicação básica de um problema clássico e conhecido, utilizando métodos de machine learnign a partir de um framework que ja nos traz uma gama de ferramentas prontas para realizar inúmeras atividades com machine learning.

### IMC

Calcular o IMC (Índice de Massa Corporal) é um exercício clássico que grande parte dos iniciantes em programação costumam desenvover. Da forma clássica, nós poderiamos escrever um algoritmo com regras condicionais fixa que realizem a validação lógica a partir de uma entrada, devolvendo então um resultado.

Por exemplo, digamos que nossas regras para definição do IMC sejam:

- Abaixo do peso: 18 Kg ou menos
- Peso ideal: de 19 a 25 Kg;
- Sobrepeso: de 26 a 30 kg;
- Obesidade: de 31 a 41 Kg;
- Obesidade mórbida: Acima de 41 Kg;

Seguindo um paradigma clássico de programação, poderiamos implementar estas regras sob uma estrutura condicional, com as regras explicitamente declaradas:

In [9]:
def imc(weight: float) -> str:
    if weight <= 18.9:
        return 'underweight'
    elif 19.0 <= weight <= 25.9:
        return 'ideal'
    elif 26.0 <= weight <= 30.9:
        return 'overweight'
    elif 31.0 <= weight <= 41.9:
        return 'obesity'
    else:
        return 'morbid'

Este algoritmo funciona como deveria funcionar, pois programamos explicitamente o algoritmo testar as condições da regra. Podemos provar a função com algumas tentativas:

In [13]:
assert imc(9.78) == 'underweight'
assert imc(28.976) == 'overweight'
assert imc(100) == 'morbid'

Essa é uma forma clássica de se resolver este problema. Ao receber um input, a função vai testar cada possibilidade, retornando o resultado caso encontre um valor verdadeiro.

Se quiséssemos utilizar uma abordagem baseada em Inteligência Artificial, mais especificamente, selecionando um modelo de machine learning, a interpretação que teriamos de tomar seria um pouco diferente.

Ao utiliza machine learning, não informaremos ao programa as regras que definimos anteriormente, vamos deixar que o algoritmo chegue à estas conclusões sozinho.

### mas como isso é possível?

Resposta: Com dados.

Os dados são o fermento natural que o machine learning precisa para fazer "crescer o seu bolo". A partir dos dados que tivermos, podemos *ajustar* um modelo para **estimar** a probabilidade de que uma determinada entrada **X** seja da categoria **y**.

Como assim?

Vamos exemplificar com o mesmo problema do IMC, porém desta vez, não iremos declarar explicitamente as regras condicionais, pelo contrário, vamos gerar varias categorias de dados dentro das faixas.

Imagine que nós tivessemo algumas listas de pesos de pessoas reais. As listas estão separadas por faixa de peso da mesma forma que as regras anteriores separam as categorias:

In [32]:
# Vamos utilizar algumas bibliotecas do Python
from random import random, randint
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [41]:
# geramos pesos (Kg) para 1000 pessoas em cada categora, somando 5000 exemplos:
underweight = [random() + randint(0, 18) - .5 for _ in range(1000)]
ideal = [random() + randint(19, 25) -.5 for _ in range(1000)]
overweight = [random() + randint(25, 30) for _ in range(1000)]
obesity = [random() + randint(30, 41) - .5 for _ in range(1000)]
morbid = [random() + randint(40, 100) for _ in range(1000)]

# Nós também geramos a correspondencia aos exemplos gerados
underweight_targets = [0 for _ in range(1000)]
ideal_targets = [1 for _ in range(1000)]
overweight_targets = [2 for _ in range(1000)]
obesity_targets = [3 for _ in range(1000)]
morbid_targets = [4 for _ in range(1000)]

## O que é essa correspondência?

O Machine Learning somente entende números, então precisamos mapear a categoria que queremos classificar par aum **número**, desta forma mapeamos um inteiro fixo para cada categoria:

- `0` -> underweight;
- `1` -> ideal;
- `2` -> overweight;
- `3` -> obesity;
- `4` -> morbid;

Desta forma, quando nosso algoritmo retornar `3` sabemos que ele classificou a entrada como **obesidade**.

Podemos escrever esse mapa com um `dict` python:

In [42]:
results = {
    0: 'underweight',
    1: 'ideal',
    2: 'overweight',
    3: 'obesity',
    4: 'morbid'
}

results.get(2)  # deve retornar sobrepeso

'overweight'

Resumindo, criamos 1000 dados na faixa de peso de peso de uma categoria, para cada categoria, e da mesma forma, criamos 1000 rótulos para cada categoria, ou seja, 1000 exemplo de peso ideal, teremos uma lista de 1000 números `1`.

Agora vamos unir estes dados, precisamos concatenar as listas, gerando uma unica lista de 5000 exemplos ordenados (ordenados pois precisamos mapear para os rótulos) e uma lista com 5000 respostas:

In [43]:
dataset = {
    'samples': np.array(low + ideal + overweight + obesity + morbid),
    'targets': underweight_targets + ideal_targets + overweight_targets + obesity_targets + morbid_targets,
}

### O que nós queremos com isso?

Nós queremos poder ensinar a máquina que cada elemento da lista `samples` corresponde ao seu indice equivalente, da lista `targets`, ou em outras palavras, queremos ensinar o computador, que o elemento em `dataset['samples'][0]` corresponde ao `dataset['targets'][0]`:

In [44]:
print('O peso (Kg): ', dataset['samples'][0], ' corresponde a ', results.get(dataset['targets'][0]))

O peso (Kg):  5.58631550444403  corresponde a  underweight


Certo, dito isso, vamos aos procedimentos de ensinar a máquina.

É muito comum em Machine Learning que os dados sejam divididos em um coinjunto de **treino** e um conjunto de **teste**.

O conjunto de **treino** é utilizado para ensinar a máquina, onde ensinar a máquina, significa que vamos mostrar ao algoritmo varios exemplos de dados e a resposta para o exemplo. Da mesma forma que, quando somos crianças, um adulto chega em nossa frente, com uma fruta avermelhada em suas mãos e nos diz: "Isto é uma maçã". Estamos a fazer a mesma coisa com o algoritmo, porém com um caminhão de maçãs, peras, melancias e pêssegos. Isto é chamado de **Aprendizado Supervisionado**.

![](https://image.slidesharecdn.com/thisisanapple-140802233157-phpapp01/95/this-is-an-apple-2-638.jpg?cb=1407022351)


O conjunto de **teste** deve representar uma  parcela de dados que que o algoritmo ainda não viu. Utilizamos este conjunto para medir a capacidade que o algoritmo possui de acertar ou errar uma pergunta que fizermos. Por exemplo, após treinar o algoritmo com varias iomagens de maçãs, peras, melacias e pêssegos, poderiamos chegar com um fruto de forma meio "arredondada" de coloração avermelhada e perguntar ao algoritmo: "O que é isso".

Como nós mostramos ao algoritmo mil exemplos de maçãs com esta característica, nosso hipótese seria de que ele respondesse: "É uma maçã."

O scikit ja implementa um método que realiza a separação dos nosso dados em um conjunto de treino e teste:

In [45]:
# Mistura os dados e separa-os em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    dataset['samples'],
    dataset['targets']
)

Para ete exemplo utilizaremos um modelo chamado KNN (K-Nearest Neighbors) ou em português: *K vizinhos mais próximos*. Este modelo é conhecido por classificar um dado baseado em outros grupos de dados com características semelhantes.

Eu sugiro que, após ler este tutorial que você leitor, busque na [documentação do scikit-learn](https://scikit-learn.org/stable/supervised_learning.html) por outros modelos e tente utilizá-los para treinar os mesmos dados e ver o que acontece.

In [46]:
# Inicializa um modelo de K-vizinhos mais próximos
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train.reshape(-1, 1), y_train)

Na execução acima nós instanciamos o estimador KNN com o parâmetro `n_neighbors` com valor 3. Isto significa que o classificador vai tentar dar a resposta par aum determinado dado de entrada, a aprtir das carcterísticas dos 3 vizinhos mais próximos deste dado.

Ou seja, se eu mostrar ao algoritmo de maçãs e pêras um dado arrendondado e vermelho, o algoritmo vai tentar lembrar de todas as frutas que ele viu, colocar este novo dado próximo de 3 outros exemplos muito parecidos, onde dois deles são maças e um deles é uma pêra. O novo dado se aproxima mais das duas maçãs, logo, o algotitmo infere que, como o novo dado se parece com as maçãs, deve ser uma maçã.

O método encadeado `.fit()` é a chamada especial que "ensina" o modelo que cada exemplo contido em `X_train` correspodne aos rótulos de `y_train`.

Podemos agora testar a precisãod e acerto do modelo com o conjunto de testes:

In [47]:
# Avalia o modelo
y_pred = knn.predict(X_test.reshape(-1, 1))
print("Test set score: {:.2f}".format(np.mean(y_pred == y_test)))

Test set score: 0.95


Nosso modelo acerta com 95% de precisão.

Mas aí você pensa "poxa mas ele não acerta 100%, então é ruim", bem isso é uma meia verdade. Em Inteligência Artificial, um algoritmo jamais chegará a 100%, e se chegar, deve estar errado. Se um modelo de machine learning acertar com 100% de precisão, isso significa que ele estará **super ajustado**, o que quer dizer que especialmente para os dados que ele conhece, ele acerta 100% das ṕerguntas, mas para dados que ele nunca viu (dados que não existem no dataset de treino) ele não terá capacidade de generalizar e acertar a resposta. Esse é um dos viéses do machine learning que nos permitem brincar com inúmeros parâmetros para ajustar os modelos.

Bem chega de blablabla e vamos testar nosso modelo com algumas entradas.

Primeiro vamos redefinir nossa função de IMC para retornar o método `predict()` do modelo KNN sobre o peso inserido. Perceba como agora não temos uma "pilha de IFs":

In [59]:
def imc(weight: float) -> str:
    return results.get(knn.predict([[weight]])[0])

In [60]:
print('14.4: ', imc(14.5))
print('17.9: ', imc(17.9))
print('18.6: ', imc(18.6))
print('22.0: ', imc(22.0))
print('28.9: ', imc(28.9))
print('34: ', imc(34))
print('89.22: ', imc(89.22))

14.4:  underweight
17.9:  underweight
18.6:  ideal
22.0:  ideal
28.9:  overweight
34:  obesity
89.22:  morbid


## Concluindo

Este é um exercío básico e prático para exemplificar como tarefas clássicas de ciência da computação podem ser facilmente implementadas com Machine Learning. A biblioteca Scikit-Learn ja nos traz vários modelos prontos para usar, facilitando muito trabalho envolvido por baixo dos panos, como cálculos estatísticos e ajustes de parâmetros internos do modelo.

Esta implementação é lúdica, dificilmente um modelo desse tamanho seria mais eficiente do que uma implementação clássica com tão poucas condições como este exemplo, mas para grandes sistemas o machine learning pode apresentar um salto enorme em eficiência computacional.